In [103]:
import requests  
# parse HTML / XML documents (Week 3)
from bs4 import BeautifulSoup 

import pandas as pd  
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json
# transform JSON file into Pandas
from pandas.io.json import json_normalize 

!pip install geocoder
import geocoder
from geopy.geocoders import Nominatim  




In [104]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Townships_of_Yangon").text
soup = BeautifulSoup(data, 'html.parser')
nlist = []

for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    nlist.append(row.text)

ygn_df = pd.DataFrame({"Townships": nlist})

ygn_df.head(10)

,Townships
0,Ahlon Township
1,Bahan Township
2,Botataung Township
3,Dagon Seikkan Township
4,Dagon Township
5,Dala Township
6,Dawbon Township
7,East Dagon Township
8,Hlaing Township
9,Hlaingthaya Township


In [105]:
def get_latlng(twsp):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Yangon, Myanmar'.format(twsp))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [106]:
coords = [get_latlng(twsp) for twsp in ygn_df["Townships"].tolist()]
coords

[[16.78806000000003, 96.12389000000007],
 [16.813150000000064, 96.15587000000005],
 [16.766670000000033, 96.16667000000007],
 [16.768490000000043, 96.16206000000005],
 [16.80528000000004, 96.15611000000007],
 [16.75917000000004, 96.15972000000005],
 [16.781940000000077, 96.18111000000005],
 [16.80528000000004, 96.15611000000007],
 [17.383330000000058, 95.83333000000005],
 [16.875630000000058, 96.06248000000005],
 [16.889170000000036, 96.10306000000008],
 [16.83000000000004, 96.12444000000005],
 [16.773030000000063, 96.16160000000008],
 [16.80583000000007, 96.12222000000008],
 [16.779810000000055, 96.14597000000003],
 [16.779580000000067, 96.15187000000003],
 [16.87000000000006, 96.13944000000004],
 [16.78776000000005, 96.17753000000005],
 [16.925000000000068, 96.09833000000003],
 [16.80528000000004, 96.15611000000007],
 [16.91303000000005, 96.16534000000007],
 [16.779440000000022, 96.15618000000006],
 [16.794440000000066, 96.17444000000006],
 [17.006940000000043, 96.08611000000008],
 [

In [107]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
ygn_df['Latitude'] = df_coords['Latitude']
ygn_df['Longitude'] = df_coords['Longitude']

print(ygn_df.shape)
ygn_df

(33, 3)


,Townships,Latitude,Longitude
0,Ahlon Township,16.78806,96.12389
1,Bahan Township,16.81315,96.15587
2,Botataung Township,16.76667,96.16667
3,Dagon Seikkan Township,16.76849,96.16206
4,Dagon Township,16.80528,96.15611
5,Dala Township,16.75917,96.15972
6,Dawbon Township,16.78194,96.18111
7,East Dagon Township,16.80528,96.15611
8,Hlaing Township,17.38333,95.83333
9,Hlaingthaya Township,16.87563,96.06248


In [108]:
ygn_df.to_csv("ygn_df.csv", index=False)

In [111]:
#df_yan = pd.DataFrame()
#df_yan = pd.read_csv("ygn_df.csv", index_col = 0)
#df_yan.reset_index(inplace=True, drop=True)
#df_yan.head()

,Latitude,Longitude
Townships,,
Ahlon Township,16.78806,96.12389
Bahan Township,16.81315,96.15587
Botataung Township,16.76667,96.16667
Dagon Seikkan Township,16.76849,96.16206
Dagon Township,16.80528,96.15611


In [112]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium

In [113]:
address = 'Yangon, Myanmar'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The coordinates of Yangon, Myanmar is {}, {}.'.format(latitude, longitude))

The coordinates of Yangon, Myanmar is 16.7967129, 96.1609916.


In [114]:
map_ygn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, twsp in zip(ygn_df['Latitude'], ygn_df['Longitude'], ygn_df['Townships']):
    label = '{}'.format(twsp)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_ygn)  
    
map_ygn

In [115]:
map_ygn.save('map_ygn.html')

<font size="4">Foursquare API to explore each township</font>

In [116]:
CLIENT_ID = 'BJ3R0NDE5FJEH0QS2U312ZLJBASZJJ50KWQZRCCV02KFL4YY'
CLIENT_SECRET = 'OZOW1V52JBASEHSZFYZ5RI1OG3DL2KX5DMPTNIAB2Y1NBEVO'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BJ3R0NDE5FJEH0QS2U312ZLJBASZJJ50KWQZRCCV02KFL4YY
CLIENT_SECRET:OZOW1V52JBASEHSZFYZ5RI1OG3DL2KX5DMPTNIAB2Y1NBEVO


<font size="3">Get the top 100 venues that are within a radius of 1000 meters.</font>

In [117]:
radius = 1000
LIMIT = 100
places = []

for lat, long, twsp in zip(ygn_df['Latitude'], ygn_df['Longitude'], ygn_df['Townships']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        places.append((
            twsp,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [118]:
places_df = pd.DataFrame(venues)

# define the column names
places_df.columns = ['Township', 'Latitude', 'Longitude', 'Name', 'Latitude', 'Longitude', 'Category']

print(venue_df.shape)
places_df.head(20)

(7, 3)


,Township,Latitude,Longitude,Name,Latitude,Longitude,Category
0,Ahlon Township,16.78806,96.12389,Mahlzeit,16.797777,96.130063,German Restaurant
1,Ahlon Township,16.78806,96.12389,Lucky Seven Tea Shop,16.783131,96.133287,Café
2,Ahlon Township,16.78806,96.12389,Hotel Grand United (Ahlone Branch),16.784748,96.135126,Hotel
3,Ahlon Township,16.78806,96.12389,Kou Fu Restaurant,16.787631,96.133878,Chinese Restaurant
4,Ahlon Township,16.78806,96.12389,Belmond Governor's Residence,16.788131,96.139003,Hotel
5,Ahlon Township,16.78806,96.12389,Shan Yoe Yar Fine Dining Restaurant,16.780792,96.139372,Burmese Restaurant
6,Ahlon Township,16.78806,96.12389,The Marina Indian Restaurant,16.790575,96.138522,Indian Restaurant
7,Ahlon Township,16.78806,96.12389,Atlas,16.797572,96.129954,Bar
8,Ahlon Township,16.78806,96.12389,Burbrit Taproom,16.791412,96.139558,Beer Bar
9,Ahlon Township,16.78806,96.12389,cocoon cafe & bar,16.796607,96.131232,Bar


In [119]:
#Check number of venues per township
places_df.groupby(["Township"]).count()

,Latitude,Longitude,Name,Latitude,Longitude,Category
Township,,,,,,
Ahlon Township,47,47,47,47,47,47
Bahan Township,100,100,100,100,100,100
Botataung Township,88,88,88,88,88,88
Dagon Seikkan Township,90,90,90,90,90,90
Dagon Township,100,100,100,100,100,100
Dala Township,51,51,51,51,51,51
Dawbon Township,42,42,42,42,42,42
East Dagon Township,100,100,100,100,100,100
Hlaingthaya Township,4,4,4,4,4,4


In [120]:
#Find out how many unique categories from all venues
print('There are {} unique categories.'.format(len(places_df['Category'].unique())))

There are 120 unique categories.


In [121]:
# print out list of categories
places_df['Category'].unique()[:50]

array(['German Restaurant', 'Café', 'Hotel', 'Chinese Restaurant',
       'Burmese Restaurant', 'Indian Restaurant', 'Bar', 'Beer Bar',
       'Japanese Restaurant', 'Thai Restaurant', 'Bakery',
       'Italian Restaurant', 'Karaoke Bar', 'Asian Restaurant',
       'Bookstore', 'Dim Sum Restaurant', 'Cantonese Restaurant',
       'Restaurant', 'French Restaurant', 'BBQ Joint', 'Platform',
       'Seafood Restaurant', 'Ramen Restaurant', 'Athletics & Sports',
       'Soup Place', 'Road', 'Spa', 'Burrito Place', 'Korean Restaurant',
       'Shopping Mall', 'Sandwich Place', 'Steakhouse', 'Gym',
       'Pizza Place', 'Tea Room', 'Noodle House', 'Buffet', 'Hostel',
       'Diner', 'Sushi Restaurant', 'Mexican Restaurant',
       'Bubble Tea Shop', 'Coffee Shop', 'Ice Cream Shop', 'Whisky Bar',
       'Lake', 'Comfort Food Restaurant', 'Flower Shop',
       'American Restaurant', 'Warehouse Store'], dtype=object)

In [122]:
ygn_onehot = pd.get_dummies(places_df[['Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ygn_onehot['Townships'] = places_df['Township'] 

# move neighborhood column to the first column
fixed_columns = [ygn_onehot.columns[-1]] + list(ygn_onehot.columns[:-1])
ygn_onehot = ygn_onehot[fixed_columns]

print(ygn_onehot.shape)
ygn_onehot.head()

(1740, 121)


,Townships,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Boarding House,Boat or Ferry,Bookstore,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Cafeteria,Café,Cantonese Restaurant,Chinese Restaurant,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cruise,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gym,Gym / Fitness Center,Harbor / Marina,History Museum,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Karaoke Bar,Korean Restaurant,Lake,Light Rail Station,Lounge,Market,Massage Studio,Mexican Restaurant,Motel,Movie Theater,Moving Target,Multiplex,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Other Great Outdoors,Park,Pier,Pizza Place,Platform,Pub,Ramen Restaurant,Resort,Restaurant,Road,Salad Place,Sandwich Place,Seafood Restaurant,Shabu-Shabu Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Train Station,Video Store,Warehouse Store,Whisky Bar
0,Ahlon Township,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Ahlon Township,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Ahlon Township,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Ahlon Township,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ahlon Township,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [123]:
ygn_grouped = ygn_onehot.groupby(["Townships"]).mean().reset_index()

print(ygn_grouped.shape)
ygn_grouped

(32, 121)


,Townships,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Boarding House,Boat or Ferry,Bookstore,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Cafeteria,Café,Cantonese Restaurant,Chinese Restaurant,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Cruise,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Flea Market,Flower Shop,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gym,Gym / Fitness Center,Harbor / Marina,History Museum,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Karaoke Bar,Korean Restaurant,Lake,Light Rail Station,Lounge,Market,Massage Studio,Mexican Restaurant,Motel,Movie Theater,Moving Target,Multiplex,Neighborhood,Nightclub,Noodle House,North Indian Restaurant,Other Great Outdoors,Park,Pier,Pizza Place,Platform,Pub,Ramen Restaurant,Resort,Restaurant,Road,Salad Place,Sandwich Place,Seafood Restaurant,Shabu-Shabu Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Spa,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Train Station,Video Store,Warehouse Store,Whisky Bar
0,Ahlon Township,0.00,0.085106,0.021277,0.000000,0.021277,0.042553,0.063830,0.000000,0.021277,0.000000,0.000000,0.000000,0.00000,0.042553,0.000000,0.000000,0.000000,0.000000,0.042553,0.000000,0.000000,0.000000,0.106383,0.021277,0.021277,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.042553,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.021277,0.000000,0.000,0.000000,0.000000,0.000000,0.021277,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.085106,0.000000,0.000000,0.000000,0.021277,0.00,0.021277,0.042553,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.021277,0.000000,0.021277,0.000000,0.042553,0.021277,0.000000,0.00,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.021277,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.063830,0.000000,0.000000,0.000000,0.000000
1,Bahan Township,0.01,0.020000,0.010000,0.000000,0.010000,0.020000,0.040000,0.000000,0.010000,0.000000,0.000000,0.000000,0.00000,0.000000,0.010000,0.010000,0.020000,0.010000,0.020000,0.020000,0.000000,0.000000,0.060000,0.000000,0.040000,0.000000,0.010000,0.010000,0.000000,0.000,0.000000,0.000000,0.000000,0.010000,0.000000,0.010000,0.020000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.010000,0.00,0.000000,0.020000,0.000000,0.000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020,0.000000,0.000000,0.000000,0.010000,0.070000,0.000000,0.010000,0.020000,0.000000,0.00,0.030000,0.050000,0.000000,0.020000,0.010000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.020000,0.000000,0.000000,0.000000,0.0,0.050000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.01,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.030000,0.00,0.000000,0.010000,0.000000,0.010000,0.060000,0.000000,0.000000,0.010000,0.010000
2,Botataung Township,0.00,0.022727,0.000000,0.000000,0.000000,0.022727,0.079545,0.022727,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.011364,0.000000,0.011364,0.011364,0.045455,0.000000,0.000000,0.000000,0.056818,0.000000,0

In [137]:
#Group the total number of shopping malls within Yangon

len(ygn_grouped[ygn_grouped["Hotel"] > 0])

25

<font size="3">Table of shopping malls</font>



In [139]:
ygn_hotel = ygn_grouped[["Townships","Hotel"]]
ygn_hotel

,Townships,Hotel
0,Ahlon Township,0.085106
1,Bahan Township,0.070000
2,Botataung Township,0.102273
3,Dagon Seikkan Township,0.133333
4,Dagon Township,0.060000
5,Dala Township,0.098039
6,Dawbon Township,0.071429
7,East Dagon Township,0.060000
8,Hlaingthaya Township,0.000000
9,Insein Township,0.000000


<font size="4">Clustering exercise</font>

In [140]:
kclusters = 5

ygn_clustering = ygn_hotel.drop(["Townships"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ygn_clustering)

kmeans.labels_[0:10]

array([4, 0, 4, 2, 0, 4, 0, 0, 1, 1], dtype=int32)

In [141]:
ygn_hotel2 = ygn_hotel.copy()
ygn_hotel2["Cluster Labels"] = kmeans.labels_
ygn_hotel2

,Townships,Hotel,Cluster Labels
0,Ahlon Township,0.085106,4
1,Bahan Township,0.070000,0
2,Botataung Township,0.102273,4
3,Dagon Seikkan Township,0.133333,2
4,Dagon Township,0.060000,0
5,Dala Township,0.098039,4
6,Dawbon Township,0.071429,0
7,East Dagon Township,0.060000,0
8,Hlaingthaya Township,0.000000,1
9,Insein Township,0.000000,1


In [142]:
#merge latitude & longitude data

ygn_hotel2 = ygn_hotel2.join(ygn_df.set_index("Townships"), on="Townships")

print(ygn_hotel2.shape)
ygn_hotel2.head(10) 

(32, 5)


,Townships,Hotel,Cluster Labels,Latitude,Longitude
0,Ahlon Township,0.085106,4,16.78806,96.12389
1,Bahan Township,0.070000,0,16.81315,96.15587
2,Botataung Township,0.102273,4,16.76667,96.16667
3,Dagon Seikkan Township,0.133333,2,16.76849,96.16206
4,Dagon Township,0.060000,0,16.80528,96.15611
5,Dala Township,0.098039,4,16.75917,96.15972
6,Dawbon Township,0.071429,0,16.78194,96.18111
7,East Dagon Township,0.060000,0,16.80528,96.15611
8,Hlaingthaya Township,0.000000,1,16.87563,96.06248
9,Insein Township,0.000000,1,16.88917,96.10306


In [143]:
# sort results by cluster Labels

ygn_hotel2.sort_values(["Cluster Labels"], inplace=True)
ygn_hotel2

,Townships,Hotel,Cluster Labels,Latitude,Longitude
15,Mayangon Township,0.050000,0,16.87000,96.13944
27,South Okkalapa Township,0.071429,0,16.85074,96.18494
26,South Dagon Township,0.060000,0,16.80528,96.15611
18,North Dagon Township,0.060000,0,16.80528,96.15611
12,Kyimyindaing Township,0.068966,0,16.80583,96.12222
10,Kamayut Township,0.034483,0,16.83000,96.12444
7,East Dagon Township,0.060000,0,16.80528,96.15611
31,Yankin Township,0.052632,0,16.83716,96.16497
1,Bahan Township,0.070000,0,16.81315,96.15587
6,Dawbon Township,0.071429,0,16.78194,96.18111


<font size="4">Visualize clusters</font>

In [130]:
import numpy as np 

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ygn_hotel2['Latitude'], ygn_hotel2['Longitude'], ygn_hotel2['Townships'], ygn_mall2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [131]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

<font size="4">Examine clusters</font>

<font size="2">Cluster 0</font>

In [132]:
ygn_mall2.loc[ygn_mall2['Cluster Labels'] == 0]

,Townships,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Ahlon Township,0.000000,0,16.78806,96.12389
18,North Dagon Township,0.010000,0,16.80528,96.15611
17,Mingaladon Township,0.000000,0,16.92500,96.09833
22,Sanchaung Township,0.000000,0,17.00694,96.08611
14,Latha Township,0.010000,0,16.77958,96.15187
13,Lanmadaw Township,0.010000,0,16.77981,96.14597
24,Seikkyi Kanaungto Township,0.000000,0,16.76850,96.11098
19,North Okkalapa Township,0.000000,0,16.91303,96.16534
9,Insein Township,0.000000,0,16.88917,96.10306
25,Shwepyitha Township,0.000000,0,16.97183,96.09478


<font size="2">Cluster 1</font>

In [133]:
ygn_mall2.loc[ygn_mall2['Cluster Labels'] == 1]

,Townships,Shopping Mall,Cluster Labels,Latitude,Longitude
16,Mingala Taungnyunt Township,0.117647,1,16.78776,96.17753
29,Thaketa Township,0.125000,1,16.80412,96.21216


<font size="2">Cluster 2</font>

In [134]:
ygn_mall2.loc[ygn_mall2['Cluster Labels'] == 2]

,Townships,Shopping Mall,Cluster Labels,Latitude,Longitude
23,Seikkan Township,0.044444,2,16.76849,96.16206
31,Yankin Township,0.026316,2,16.83716,96.16497
12,Kyimyindaing Township,0.034483,2,16.80583,96.12222
11,Kyauktada Township,0.040000,2,16.77303,96.16160
10,Kamayut Township,0.051724,2,16.83000,96.12444
3,Dagon Seikkan Township,0.044444,2,16.76849,96.16206
2,Botataung Township,0.045455,2,16.76667,96.16667


<font size="2">Cluster 3</font>

In [135]:
ygn_mall2.loc[ygn_mall2['Cluster Labels'] == 3]

,Townships,Shopping Mall,Cluster Labels,Latitude,Longitude
30,Thingangyun Township,0.181818,3,16.82778,96.1975


<font size="2">Cluster 4</font>

In [136]:
ygn_mall2.loc[ygn_mall2['Cluster Labels'] == 4]

,Townships,Shopping Mall,Cluster Labels,Latitude,Longitude
6,Dawbon Township,0.095238,4,16.78194,96.18111
28,Tamwe Township,0.078431,4,16.81000,96.17778
21,Pazundaung Township,0.067797,4,16.79444,96.17444
15,Mayangon Township,0.075000,4,16.87000,96.13944
